<a href="https://colab.research.google.com/github/denisangelo/Federated_Learning/blob/main/Aula_FL_sem_Divisao_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
#Importanto Pytorch, Flower,Opacus
!pip install -q flwr[simulation]
!pip install torch
!pip install torchvision
!pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
from collections import OrderedDict
from typing import List, Tuple
import matplotlib.pyplot as plt

import flwr as fl
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from flwr.common import Metrics
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

In [25]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE} using Pytorch{torch.__version__}, Flower{fl.__version__}" )

Training on cpu using Pytorch2.0.1+cu118, Flower1.4.0


In [26]:
BATCH_SIZE = 32
NUM_CLIENTS= 2
optim_lr=0.001
local_epochs=1
num_rounds=5

In [27]:
def load_data():
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize( (0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    trainset = CIFAR10(root="./data", train=True,download=True, transform=transform)
    testset = CIFAR10(root="./data", train=False, download=True, transform=transform)
    trainloader = DataLoader(trainset, batch_size=BATCH_SIZE,shuffle=True,num_workers=0)
    testloader = DataLoader(testset,batch_size=BATCH_SIZE,shuffle=False,num_workers=0)
    num_examples = {"trainset": len(trainset)*BATCH_SIZE, "testset": len(testset)*BATCH_SIZE}

    return trainloader, testloader, num_examples

In [28]:
# Definindo o Modelo
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [29]:
# Definindo as funções de treinamento
def train(net, trainloader, epochs: int, verbose=False):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(),lr=optim_lr)
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        if verbose:
            print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [30]:
# Funções de atualização de parâmetros do modelo
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

In [31]:
# Definindo a classe de cliente Virtual
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, testloader,num_examples) -> None:
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.testloader = testloader
        self.num_examples = num_examples

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=local_epochs)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.testloader)
        return float(loss), len(self.testloader), {"accuracy": float(accuracy)}

def client_fn(cid) -> FlowerClient:
        net = Net().to(DEVICE)
        trainloader, testloader, num_examples = load_data()
        return FlowerClient(cid, net, trainloader, testloader,num_examples)

In [32]:
# Realizando a agregação
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

In [33]:

# Iniciando o treino federado
# Criando a Função de agregação FedAvg
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,#-->Fração de clientes necessários para treinamento
    fraction_evaluate=1.0, #--> Fração de clientes que serão utilizados para teste
    min_fit_clients=NUM_CLIENTS, #--> Número mínimo que serão utlilizados para treinamento
    min_evaluate_clients=2, #--> Número mínimo de clientes que serão realizados para teste
    min_available_clients=NUM_CLIENTS,#--> Número mínimo de clientes que serão utilizados no treinamento
    evaluate_metrics_aggregation_fn=weighted_average,  
)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}
# Inicia a simulação
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=num_rounds),
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-06-08 11:41:17,041 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)


(launch_and_evaluate pid=35847) Files already downloaded and verified [repeated 3x across cluster]
(launch_and_evaluate pid=35847) [Client 0] evaluate, config: {}


2023-06-08 11:41:22,198	INFO worker.py:1625 -- Started a local Ray instance.
INFO flwr 2023-06-08 11:41:26,623 | app.py:180 | Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'object_store_memory': 3920279961.0, 'CPU': 2.0, 'memory': 7840559924.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'object_store_memory': 3920279961.0, 'CPU': 2.0, 'memory': 7840559924.0}
INFO flwr 2023-06-08 11:41:26,628 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-06-08 11:41:26,630 | server.py:273 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client
(pid=39895) 2023-06-08 11:41:35.108517: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(launch_and_get_parameters pid=39895) Files already downloaded and verified
(launch_and_get_parameters pid=39895) Files already downloaded and verified


INFO flwr 2023-06-08 11:41:41,302 | server.py:277 | Received initial parameters from one random client
INFO:flwr:Received initial parameters from one random client
INFO flwr 2023-06-08 11:41:41,313 | server.py:88 | Evaluating initial parameters
INFO:flwr:Evaluating initial parameters
INFO flwr 2023-06-08 11:41:41,318 | server.py:101 | FL starting
INFO:flwr:FL starting
DEBUG flwr 2023-06-08 11:41:41,322 | server.py:218 | fit_round 1: strategy sampled 2 clients (out of 2)
DEBUG:flwr:fit_round 1: strategy sampled 2 clients (out of 2)


...

(launch_and_fit pid=39895) Files already downloaded and verified [repeated 4x across cluster]
(launch_and_evaluate pid=39893) Files already downloaded and verified
(launch_and_fit pid=39895) [Client 1] fit, config: {}


DEBUG flwr 2023-06-08 11:49:02,026 | server.py:232 | fit_round 5 received 2 results and 0 failures
DEBUG:flwr:fit_round 5 received 2 results and 0 failures
DEBUG flwr 2023-06-08 11:49:02,051 | server.py:168 | evaluate_round 5: strategy sampled 2 clients (out of 2)
DEBUG:flwr:evaluate_round 5: strategy sampled 2 clients (out of 2)


(launch_and_evaluate pid=39895) Files already downloaded and verified [repeated 4x across cluster]
(launch_and_fit pid=39893) Files already downloaded and verified
(launch_and_evaluate pid=39895) [Client 0] evaluate, config: {}


DEBUG flwr 2023-06-08 11:49:14,831 | server.py:182 | evaluate_round 5 received 2 results and 0 failures
DEBUG:flwr:evaluate_round 5 received 2 results and 0 failures
INFO flwr 2023-06-08 11:49:14,835 | server.py:147 | FL finished in 453.5137521709994
INFO:flwr:FL finished in 453.5137521709994
INFO flwr 2023-06-08 11:49:14,843 | app.py:218 | app_fit: losses_distributed [(1, 0.048354385781288145), (2, 0.0370848767220974), (3, 0.03386085147857666), (4, 0.03235412722229958), (5, 0.031617751121520994)]
INFO:flwr:app_fit: losses_distributed [(1, 0.048354385781288145), (2, 0.0370848767220974), (3, 0.03386085147857666), (4, 0.03235412722229958), (5, 0.031617751121520994)]
INFO flwr 2023-06-08 11:49:14,845 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO:flwr:app_fit: metrics_distributed_fit {}
INFO flwr 2023-06-08 11:49:14,848 | app.py:220 | app_fit: metrics_distributed {'accuracy': [(1, 0.4481), (2, 0.5749), (3, 0.615), (4, 0.6307), (5, 0.6403)]}
INFO:flwr:app_fit: metrics_distributed 

History (loss, distributed):
	round 1: 0.048354385781288145
	round 2: 0.0370848767220974
	round 3: 0.03386085147857666
	round 4: 0.03235412722229958
	round 5: 0.031617751121520994
History (metrics, distributed, evaluate):
{'accuracy': [(1, 0.4481), (2, 0.5749), (3, 0.615), (4, 0.6307), (5, 0.6403)]}